In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv("final_data.csv")


In [3]:
# Sélectionner les colonnes de fonctionnalités et la colonne cible
X = data.drop(['reordered'], axis=1)  # Supprimer la colonne cible pour les fonctionnalités
y = data['reordered']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [7]:
!pip install -U jax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.8 MB/s eta 0:00:00
  Attempting uninstall: jax
    Found existing installation: jax 0.4.23
    Uninstalling jax-0.4.23:
      Successfully uninstalled jax-0.4.23


In [8]:
import jax
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

In [10]:
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','f1'])


In [12]:
pip install scikeras

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from scikeras.wrappers import KerasClassifier


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Définition de la fonction f1_score
def f1_score(y_true, y_pred):
    y_pred = tf.math.round(y_pred)  # Arrondir les prédictions à 0 ou 1
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'), axis=0)
    tn = tf.reduce_sum(tf.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + tf.keras.backend.epsilon())
    r = tp / (tp + fn + tf.keras.backend.epsilon())

    f1 = 2 * p * r / (p + r + tf.keras.backend.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return tf.reduce_mean(f1)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

def create_full_model(filters, kernel_size, dropout_rate, optimizer_name, learning_rate):
    model = Sequential([
        Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    # Sélection et configuration de l'optimiseur
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError("Optimiseur non supporté")

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', f1_score])
    return model

In [16]:
filters_options = [32, 64]
kernel_size_options = [2, 3]
dropout_options = [0.25, 0.5]
optimizers = ['adam', 'sgd', 'rmsprop']
learning_rates = [0.001, 0.0001, 0.01]

results = []

for filters in filters_options:
    for kernel_size in kernel_size_options:
        for dropout_rate in dropout_options:
            for optimizer in optimizers:
                for lr in learning_rates:
                    print(f"Entraînement avec {filters} filtres, kernel_size={kernel_size}, dropout_rate={dropout_rate}, optimizer={optimizer}, lr={lr}")
                    model = create_full_model(filters=filters, kernel_size=kernel_size, dropout_rate=dropout_rate, optimizer_name=optimizer, learning_rate=lr)

                    history = model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2, batch_size=32, verbose=0)
                    score = model.evaluate(X_test_scaled, y_test, verbose=0)
                    # Assurez-vous que l'index de score[2] correspond bien à votre score F1 si vous l'avez inclus dans les métriques du modèle
                    results.append((filters, kernel_size, dropout_rate, optimizer, lr, score[2]))
                    print(f"Résultat pour {filters} filtres, kernel_size={kernel_size}, dropout_rate={dropout_rate}, optimizer={optimizer}, lr={lr}: F1-Score = {score[2]:.4f}")

# Affichage du résumé des résultats
print("\nRésumé des résultats:")
for result in results:
    print("Filtres: {}, Taille de noyau: {}, Taux de dropout: {}, Optimiseur: {}, LR: {}, F1-Score: {:.4f}".format(result[0], result[1], result[2], result[3], result[4], result[5]))

Entraînement avec 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=adam, lr=0.001
Résultat pour 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=adam, lr=0.001: F1-Score = 0.8853
Entraînement avec 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=adam, lr=0.0001
Résultat pour 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=adam, lr=0.0001: F1-Score = 0.8829
Entraînement avec 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=adam, lr=0.01
Résultat pour 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=adam, lr=0.01: F1-Score = 0.8853
Entraînement avec 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=sgd, lr=0.001
Résultat pour 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=sgd, lr=0.001: F1-Score = 0.8817
Entraînement avec 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=sgd, lr=0.0001
Résultat pour 32 filtres, kernel_size=2, dropout_rate=0.25, optimizer=sgd, lr=0.0001: F1-Score = 0.8809
Entraînement avec 32 filtres, kernel_